# Чтение данных

In [1]:
from src.models import Request # Датакласс для входных данных
from src.models import Coordinates
from src.antcolony import AntAlgorithm
import src.visualisation as visualisation
moscow = Coordinates(55.75222, 37.61556)
ekaterenburg = Coordinates(56.838008, 60.597452)

In [2]:
r = Request.from_txt("data/input1.txt")
r.move_to(ekaterenburg)

print(f'Capacity: {r.capacity}\nTime limit: {r.time}\nTotal points: {r.points_number}')

Capacity: 50
Time limit: 29591
Total points: 656


In [3]:
ant_algo = AntAlgorithm(
    r, n_ants=100, n_iterations=30, alpha=1, beta=1, evaporation_rate=0.5
)

In [4]:
route = ant_algo.run()
r.score(route)

83550.0

In [5]:
visualisation.GeoMap(r, route=route)